In [18]:
import pandas as pd
import numpy as np

In [19]:
data = pd.read_csv("log.csv")
data = data.rename(columns=lambda x: x.strip())
data = data.drop(["start_time", "end_time", "prompt"], axis=1)

In [20]:
data = data.loc[data["success"] == 1]
data = data.drop(["success", "task_class", "task_index"], axis=1)
data = data.assign(cost=data["mouse_distance"] / 952 + data["mouse_click"] * 0.1 + data["keyboard_press"] * 0.2)
data

,subject_id,task_round,mouse_distance,mouse_click,keyboard_press,duration,cost
0,9,0,6282,30,28,72.235,15.198739
1,9,0,9917,18,48,62.757,21.817017
2,9,0,4208,13,1,36.350,5.920168
3,9,0,9320,8,0,33.527,10.589916
4,9,0,12359,14,14,46.989,17.182143
...,...,...,...,...,...,...,...
520,202,1,5333,6,18,152.216,9.801891
521,202,1,6747,2,1,27.440,7.487185
522,202,1,738,0,4,49.442,1.575210
523,202,1,1161,3,3,48.944,2.119538


In [21]:
grouped = data.groupby(["subject_id", "task_round"], as_index=False)

In [22]:
group_data = grouped.sum()
print(group_data.columns)
# group_data = group_data.drop(["task_class", "tesk_index"], axis=1)  
group_data = round(group_data, 3)
group_data.to_csv("stat_origin.csv", index=False)

Index(['subject_id', 'task_round', 'mouse_distance', 'mouse_click',
       'keyboard_press', 'duration', 'cost'],
      dtype='object')


In [23]:
overall_data = group_data.groupby("task_round").mean()
overall_data

,subject_id,mouse_distance,mouse_click,keyboard_press,duration,cost
task_round,,,,,,
0,64.571429,175176.190476,337.904762,346.571429,819.802667,287.113286
1,64.571429,20160.380952,15.380952,1032.285714,445.878048,229.172095


In [24]:
overall_std = group_data.groupby("task_round").std()
overall_std

,subject_id,mouse_distance,mouse_click,keyboard_press,duration,cost
task_round,,,,,,
0,90.118573,52999.568469,72.466478,114.166357,192.178417,65.43831
1,90.118573,12146.673650,12.974884,450.048902,137.481902,99.71532


In [25]:
overall_data.describe()

,subject_id,mouse_distance,mouse_click,keyboard_press,duration,cost
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
mean,64.571429,97668.285714,176.642857,689.428571,632.840357,258.142690
std,0.000000,109612.730105,228.058773,484.873221,264.404634,40.970609
min,64.571429,20160.380952,15.380952,346.571429,445.878048,229.172095
25%,64.571429,58914.333333,96.011905,518.000000,539.359202,243.657393
50%,64.571429,97668.285714,176.642857,689.428571,632.840357,258.142690
75%,64.571429,136422.238095,257.273810,860.857143,726.321512,272.627988
max,64.571429,175176.190476,337.904762,1032.285714,819.802667,287.113286


In [26]:
import pandas as pd
import numpy as np
from scipy import stats

def summary(df):
    summary_df = pd.DataFrame(columns=['column_name', 'sample_size', 'mean', 'std', 'std_error', 'ci_lower', 'ci_upper', 'min', 'max'])
    dfs = []
    
    for column in df.columns:
        data = df[column]
        n = len(data)
        mean = np.mean(data)
        std = np.std(data)
        std_error = std / np.sqrt(n)
        ci = stats.t.interval(0.95, n-1, loc=mean, scale=std_error)
        ci_lower = ci[0]
        ci_upper = ci[1]

        minimum = np.min(data)
        maximum = np.max(data)
        
        dfs.append(pd.DataFrame({'column_name': column, 'sample_size': n, 'mean': mean, 
                                 'std': std, 'std_error': std_error, 'ci_lower': ci_lower, 
                                 'ci_upper': ci_upper, 'min': minimum, 'max': maximum}, index=[0]))
    
    summary_df = pd.concat(dfs, ignore_index=True)
    
    return summary_df

In [27]:
res = group_data
describe_0 = summary(res.loc[res["task_round"] == 0].drop(["subject_id", "task_round"], axis=1))

In [28]:
describe_1 =  summary(res.loc[res["task_round"] == 1].drop(["subject_id", "task_round"], axis=1))

In [29]:
describe_all = pd.concat([describe_0.assign(task_round=0), describe_1.assign(task_round=1)], ignore_index=True)

In [30]:
describe_all = describe_all.reindex(columns=['task_round', 'column_name', 'sample_size', 'mean', 'std', 'std_error', 'ci_lower', 'ci_upper', 'min', 'max'])

In [31]:
describe_all

,task_round,column_name,sample_size,mean,std,std_error,ci_lower,ci_upper,min,max
0,0,mouse_distance,21,175176.190476,51722.282735,11286.727417,151632.489646,198719.891306,98043.000,269443.000
1,0,mouse_click,21,337.904762,70.720041,15.432378,305.713385,370.096139,228.000,508.000
2,0,keyboard_press,21,346.571429,111.414956,24.312737,295.855948,397.286909,47.000,580.000
3,0,duration,21,819.802667,187.546931,40.926096,734.432327,905.173006,474.630,1143.378
4,0,cost,21,287.113286,63.861251,13.935667,258.043993,316.182579,202.308,406.428
5,1,mouse_distance,21,20160.380952,11853.939701,2586.741712,14764.532293,25556.229612,3804.000,53879.000
6,1,mouse_click,21,15.380952,12.662190,2.763116,9.617192,21.144712,2.000,59.000
7,1,keyboard_press,21,1032.285714,439.202756,95.841899,832.363016,1232.208412,28.000,2372.000
8,1,duration,21,445.878048,134.168598,29.277988,384.805234,506.950861,253.813,794.629
9,1,cost,21,229.172095,97.312188,21.235260,184.876118,273.468072,26.671,536.896


In [32]:
describe_all.to_csv("describe_all_stat.csv", index=False)